In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from pydataset import data

Paper by: Hadley Wickham

https://vita.had.co.nz/papers/tidy-data.pdf

### Data cleaning:

tidy data != clean data


- outlier checking  
- date parsing
- missing value imputation etc.
- <ins>data tidying: structuring datasets to facilitate analysis.</ins>



### Data semantics: tidy data

- Value: every value belongs to a variable and an observation.
- Variable: a variable contains all values that measure the same underlying attribute (like height, temperature, duration) across units.
- Observation: an observation contains all values measured on the same unit (like a person, or a day, or a race) across attributes.

####  datasets for lesson and exercises:

- https://classroom.google.com/c/MjU0ODgwOTM0NTgx/m/MzQzMzM2NzU0Mzc2/details

In [3]:
# lets look at this data:
treatments = pd.read_csv('untidy-data/treatment.csv')
treatments

,Unnamed: 0,treatmenta,treatmentb,treatmentc
0,John Smith,NaN,2,0
1,Jane Doe,16.0,11,3
2,Mary Johnson,3.0,1,4


In [4]:
#rename columns
treatments.columns = ['name', 'a', 'b', 'c']
treatments

,name,a,b,c
0,John Smith,NaN,2,0
1,Jane Doe,16.0,11,3
2,Mary Johnson,3.0,1,4


What is an obervation here?  
variables?  
values?  

In [5]:
# restructure data using 'melt'

treatments = treatments.melt(id_vars=['name'], var_name='treatment', value_name='response')

In [6]:
treatments

,name,treatment,response
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0
6,John Smith,c,0.0
7,Jane Doe,c,3.0
8,Mary Johnson,c,4.0


### Tidy data : 
- Each variable forms a column.
- Each observation forms a row.
- Each cell has a single value.
- data is tabular, i.e. made up of rows and columns

In [7]:
# Examples of tidy-data?

tips = data('tips')
tips.head()


,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


#### General Ideas  
- If the units are the same, maybe they should be in the same column.
- If one column has measurements of different units, it should be spread out  
- Should you be able to groupby some of the columns? combine them  
- Can I pass this data to seaborn?  

- Can we ask interesting questions and answer them with a groupby? i.e. generally we don't want to be taking row or column averages.


### How to deal with 'messy' data


##### Reshaping data:  
  
- Wide data --> Long data format (Melt)  
- Long data --> Wide Data format (pivot_table, unstack)  

#### 1. Messy data: Column headers are values, not variable names.

In [8]:
df = pd.read_csv('untidy-data/pew.csv')

In [9]:
# look at info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   religion            18 non-null     object
 1   <$10k               18 non-null     int64 
 2   $10-20k             18 non-null     int64 
 3   $20-30k             18 non-null     int64 
 4   $30-40k             18 non-null     int64 
 5   $40-50k             18 non-null     int64 
 6   $50-75k             18 non-null     int64 
 7   $75-100k            18 non-null     int64 
 8   $100-150k           18 non-null     int64 
 9   >150k               18 non-null     int64 
 10  Don't know/refused  18 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 1.7+ KB


In [10]:
# look at the head
df.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116


In [11]:
# Melt the data. Creates two new columns with deault name of 'variable' and 'value'

pd.melt(df, id_vars='religion')

,religion,variable,value
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
...,...,...,...
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8


In [12]:
# melt data and specify names of new columns

df_tidy = pd.melt(df, id_vars='religion', 
                  var_name = 'income', 
                  value_name = 'count')

df_tidy

,religion,income,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
...,...,...,...
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8


#### pd.melt arguments
- id_vars = columns you want to keep (not melt)
- var_name = name of new column you created by melting columns
- value_name = column name for values

#### Another example: one variable stored across multiple columns

In [13]:
billboard = pd.read_csv('untidy-data/billboard.csv')

In [15]:
billboard.head()

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
billboard.shape

(317, 81)

In [19]:
# melt the data

billboard_melt = pd.melt(billboard, id_vars=['year', 'artist', 'track', 'time', 'date.entered'],
                                            var_name = 'week', 
                                            value_name = 'ratings')
billboard_melt

,year,artist,track,time,date.entered,week,ratings
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk1,87.0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,wk1,91.0
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,wk1,81.0
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76.0
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,wk1,57.0
...,...,...,...,...,...,...,...
24087,2000,Yankee Grey,Another Nine Minutes,3:10,2000-04-29,wk76,NaN
24088,2000,"Yearwood, Trisha",Real Live Woman,3:55,2000-04-01,wk76,NaN
24089,2000,Ying Yang Twins,Whistle While You Tw...,4:19,2000-03-18,wk76,NaN
24090,2000,Zombie Nation,Kernkraft 400,3:30,2000-09-02,wk76,NaN


In [20]:
# what is the mean rating (across all weeks) of each track?
billboard_melt.groupby('track').ratings.mean()

track
(Hot S**t) Country G...    30.882353
3 Little Words             94.444444
911                        60.000000
A Country Boy Can Su...    86.666667
A Little Gasoline          89.833333
                             ...    
You Won't Be Lonely ...    88.692308
You'll Always Be Lov...    76.842105
You're A God               37.000000
Your Everything            72.000000
www.memory                 63.400000
Name: ratings, Length: 316, dtype: float64

#### 2. Messy data: Multiple variables are stored in one column.

In [21]:
df = pd.DataFrame({
    'name': ['Sally', 'Jane', 'Billy', 'Suzy'],
    'pet': ['dog: max', 'dog: buddy', 'cat: grizabella', 'hamster: fred']
})
df

,name,pet
0,Sally,dog: max
1,Jane,dog: buddy
2,Billy,cat: grizabella
3,Suzy,hamster: fred


In [24]:
'dog: max'.split(':')[0]

'dog'

In [26]:
df.pet.str.split(':')

0           [dog,  max]
1         [dog,  buddy]
2    [cat,  grizabella]
3      [hamster,  fred]
Name: pet, dtype: object

In [29]:
#df.pet.str.split(':').str[0] #this works but we're going to approach it a different way

df[['pet', 'pet_name']] = df.pet.str.split(':', expand = True)

In [30]:
df.head()

,name,pet,pet_name
0,Sally,dog,max
1,Jane,dog,buddy
2,Billy,cat,grizabella
3,Suzy,hamster,fred


#### Messy data: Variables are stored in both rows and columns

In [31]:
weather = pd.read_csv('untidy-data/weather.csv')

weather.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# melt the 'days'

weather_long = weather.melt(id_vars = ['id', 'year', 'month', 'element'],
                           var_name = 'day',
                           value_name = 'temperature')
weather_long

,id,year,month,element,day,temperature
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN
...,...,...,...,...,...,...
677,MX17004,2010,10,tmin,d31,NaN
678,MX17004,2010,11,tmax,d31,NaN
679,MX17004,2010,11,tmin,d31,NaN
680,MX17004,2010,12,tmax,d31,NaN


In [35]:
# pivot the element column. Reset index to go from multi-index to flat dataframe.

weather_tidy = weather_long.pivot_table(index = ['id', 'year', 'month', 'day'],
                                       columns = 'element', 
                                       values = 'temperature', 
                                       aggfunc = 'mean').reset_index()
weather_tidy

element,id,year,month,day,tmax,tmin
0,MX17004,2010,1,d30,27.8,14.5
1,MX17004,2010,2,d11,29.7,13.4
2,MX17004,2010,2,d2,27.3,14.4
3,MX17004,2010,2,d23,29.9,10.7
4,MX17004,2010,2,d3,24.1,14.4
5,MX17004,2010,3,d10,34.5,16.8
6,MX17004,2010,3,d16,31.1,17.6
7,MX17004,2010,3,d5,32.1,14.2
8,MX17004,2010,4,d27,36.3,16.7
9,MX17004,2010,5,d27,33.2,18.2


#### pd.pivot_table arguments
- Index = columns you want to keep (not pivot)
- columns = column you want to pivot
- values = values we want to populate in the new columns
- aggfunct = how you want to aggregate the duplicate rows

## Mini Exercise:

- read in excel file named dem_score.xls
- convert the data in tidy format (hint: melt the data)
- convert the melted dataframe back in wide-format (hint: pivot the data)




In [36]:
# read in the excel file named dem_score.xlsx
df = pd.read_excel('untidy-data/dem_score.xlsx')
df.head()

,country,1952,1957,1962,1967,1972,1977,1982,1987,1992
0,Albania,-9,-9,-9,-9,-9,-9,-9,-9,5
1,Argentina,-9,-1,-1,-9,-9,-9,-8,8,7
2,Armenia,-9,-7,-7,-7,-7,-7,-7,-7,7
3,Australia,10,10,10,10,10,10,10,10,10
4,Austria,10,10,10,10,10,10,10,10,10


In [37]:
# is the data in tidy format? What is the shape of data?
df.shape

(96, 10)

In [46]:
# convert dataframe in 'tidy' format (hint: pd.melt)
df_long = pd.melt(df, id_vars=['country'],
                        var_name = 'year', 
                        value_name = 'dem_score')
df_long

,country,year,dem_score
0,Albania,1952,-9
1,Argentina,1952,-9
2,Armenia,1952,-9
3,Australia,1952,10
4,Austria,1952,10
...,...,...,...
859,United Kingdom,1992,10
860,United States,1992,10
861,Uruguay,1992,10
862,Uzbekistan,1992,-9


In [47]:
# check shape of tidy dataframe
df_long.shape

(864, 3)

In [63]:
# convert the data back in wide format (hint: pivot_table)

df_wide = df_long.pivot_table(index = ['country'], 
                                       columns = 'year',
                                       values = 'dem_score').reset_index()
df_wide                                       

year,country,1952,1957,1962,1967,1972,1977,1982,1987,1992
0,Albania,-9,-9,-9,-9,-9,-9,-9,-9,5
1,Argentina,-9,-1,-1,-9,-9,-9,-8,8,7
2,Armenia,-9,-7,-7,-7,-7,-7,-7,-7,7
3,Australia,10,10,10,10,10,10,10,10,10
4,Austria,10,10,10,10,10,10,10,10,10
5,Azerbaijan,-9,-7,-7,-7,-7,-7,-7,-7,1
6,Belarus,-9,-7,-7,-7,-7,-7,-7,-7,7
7,Belgium,10,10,10,10,10,10,10,10,10
8,Bhutan,-10,-10,-10,-10,-10,-10,-10,-10,-10
9,Bolivia,-4,-3,-3,-4,-7,-7,8,9,9


#### Bit more complex example

In [50]:
sales = pd.read_csv('untidy-data/sales.csv')

In [51]:
sales.head()

,Product,2016 Sales,2016 PPU,2017 Sales,2017 PPU,2018 Sales,2018 PPU
0,A,673,5,231,7,173,9
1,B,259,3,748,5,186,8
2,C,644,3,863,5,632,5
3,D,508,9,356,11,347,14


In [53]:
# first we melt all the columns expect 'Product'

sales_melt = sales.melt(id_vars = 'Product')

sales_melt

,Product,variable,value
0,A,2016 Sales,673
1,B,2016 Sales,259
2,C,2016 Sales,644
3,D,2016 Sales,508
4,A,2016 PPU,5
5,B,2016 PPU,3
6,C,2016 PPU,3
7,D,2016 PPU,9
8,A,2017 Sales,231
9,B,2017 Sales,748


In [54]:
# shape of melted dataset
sales_melt.shape

(24, 3)

In [58]:
# here we split the 'variable' column to create two new columns 'year' and 'measure'

sales_melt[['year', 'measure']] = sales_melt.variable.str.split(expand = True)
sales_melt.head()

,Product,variable,value,year,measure
0,A,2016 Sales,673,2016,Sales
1,B,2016 Sales,259,2016,Sales
2,C,2016 Sales,644,2016,Sales
3,D,2016 Sales,508,2016,Sales
4,A,2016 PPU,5,2016,PPU


In [60]:
# now we will pivot the 'measure' column to create the two new columns 'PPU' and 'Sales'
# reset the index to flatten the dataframe 

sales_tidy = sales_melt.pivot_table(index = ['Product', 'year'], 
                                   columns = 'measure',
                                   values = 'value').reset_index()

In [61]:
sales_tidy

measure,Product,year,PPU,Sales
0,A,2016,5,673
1,A,2017,7,231
2,A,2018,9,173
3,B,2016,3,259
4,B,2017,5,748
5,B,2018,8,186
6,C,2016,3,644
7,C,2017,5,863
8,C,2018,5,632
9,D,2016,9,508
